In [ ]:
from Models.FakeModel import Model
from Models.FLAN_T5 import Flan

# import numpy as np
from checklist.editor import Editor
# from checklist.perturb import Perturb
# from checklist.test_types import MFT, INV, DIR
# from checklist.pred_wrapper import PredictorWrapper

import csv

In [ ]:
editor = Editor();

# Select one of the cells below to choose which model to test

## FakeModel

In [ ]:
# model = Model("test")

# res = model()
# print(res)

## FLAN-T5

In [ ]:
model = Flan()
filename = "FLAN_T5_tests.csv"
# res = model.get_results("test")
# print(res)

## Bloom

In [ ]:
# TBD

## Roberta

In [ ]:
# TBD

## Valid Definitions

In [ ]:
PE_ACCEPTABLE = [
    "energy at rest",
    "energy that a body has because of its position relative to other bodies",
    "energy stored in the car at the top of the initial drop",
]
PE_UNACCEPTABLE = [
    "energy in motion",
    "energy lost as the car goes down the hill",
    "opposite of kinetic energy",
]
PE_INSUFFICIENT = [
    "4 Joules at the top of the roller coaster",
    "measured in Joules",
    "equal to m*h*9.8",
]

KE_ACCEPTABLE = [
    "energy in motion",
    "energy that the car has because it is moving",
    "the work needed to accelerate the roller coaster car from rest",
]
KE_UNACCEPTABLE = [
    "energy at rest",
    "never lost nor gained as the car moves",
    "the opposite of potantial energy",
]
KE_INSUFFICIENT = [
    "4 Joules at the bottom of the roller coaster",
    "measured in Joules",
    "equal to .5*m*v^2",
]

LCE_ACCEPTABLE = [
    "energy cannot be created or destroyed, only transformed",
    "the total energy of an isolated system remains constant",
    "energy can be converted from one form to another, but never created or destroyed",
]
LCE_UNACCEPTABLE = [
    "energy can be created or destroyed, not transformed",
    "the total energy of an isolated system will change",
]
LCE_INSUFFICIENT = [
    "K1 + U1 = K2 + U2",
    "potential energy transforms into kinetic energy",
    "As the car goes down the hill, some energy is lost to friction as heat",
]


## Function for getting data and frequencies

In [ ]:
def getData(samples, expected):
    fails = 0
    data = [{"sample": "", "expected" : {}, "actual": {}, "results": {}} for _ in range(len(samples.data))]
    for i, sample in enumerate(samples.data):
        data[i]["sample"] = sample
        data[i]["expected"] = expected
        d = model.get_results(sample)
        data[i]["actual"] = d
        label = d['PE'] if len(d['PE']) != 0 else d['KE'] if len(d['KE']) != 0 else d['LCE']
        score = 0
        if expected['PE'] == d['PE']:
            score += 1
            data[i]["results"]["PE"] = "PASS"
        else:
            data[i]["results"]["PE"] = "FAIL"
        if expected['KE'] == d['KE']:
            score += 1
            data[i]["results"]["KE"] = "PASS"
        else:
            data[i]["results"]["KE"] = "FAIL"
        if expected['LCE'] == d['LCE']:
            score += 1
            data[i]["results"]["LCE"] = "PASS"
        else:
            data[i]["results"]["LCE"] = "FAIL"
            
        if score != 3:
            print("FAIL: " + sample)
            print(" Actual: " + str(d))
            print(" Expected: " + str(expected))
            data[i]["results"]["Overall"] = "FAIL"
            fails += 1
        else:
            print("PASS: " + sample)
            print(" Actual: " + str(d))
            print(" Expected: " + str(expected))
            data[i]["results"]["Overall"] = "PASS"
        
    print(str(fails) + " fails out of " + str(len(samples.data)) + " tests")
    return data


In [ ]:
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([["Test Category", "Test Type", "Test Description", "Sample",
                       "Expected PE", "Expected KE", "Expected LCE",
                       "Actual PE", "Actual KE,", "Actual LCE",
                       "PE Result", "KE Result", "LCE Result", "Overall Result"]])
    print("Successfully created file")

def writeData(category, type, description, data):
    rows = [[] for _ in range(len(data))]
    for i in range(len(data)):
        rows[i] = ([category,
                    type,
                   description,
                   data[i]["sample"],
                   data[i]["expected"]["PE"],
                   data[i]["expected"]["KE"],
                   data[i]["expected"]["LCE"],
                   data[i]["actual"]["PE"],
                   data[i]["actual"]["KE"],
                   data[i]["actual"]["LCE"],
                   data[i]["results"]["PE"],
                   data[i]["results"]["KE"],
                   data[i]["results"]["LCE"],
                   data[i]["results"]["Overall"]])
        
        
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)
    return

# Knowledge

## Minimum Functionality Tests
### Check for acceptable definitions
Should return acceptable

In [ ]:
samples = editor.template(
    "Potential energy is {pe_acceptable}", 
    pe_acceptable=PE_ACCEPTABLE
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_acceptable}.",
    lce_acceptable=LCE_ACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Knowledge", "MFT", "Correct definitions", data)
# print(data)

### Check for unacceptable definitions
should return unacceptable

In [ ]:
samples = editor.template(
    "Potential energy is {pe_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE
)
data = getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_unacceptable}",
    ke_unacceptable=KE_UNACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_unacceptable}.",
    lce_unacceptable=LCE_UNACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

writeData("Knowledge", "MFT", "Incorrect definitions", data)

### Check insufficient definitions
should return insufficient

In [ ]:
samples = editor.template(
    "Potential energy is {pe_insufficient}",
    pe_insufficient=PE_INSUFFICIENT
)
data = getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_insufficient}",
    ke_insufficient=KE_INSUFFICIENT
)
data += getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_insufficient}.",
    lce_insufficient=LCE_INSUFFICIENT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})

writeData("Knowledge", "MFT", "Insufficient definitions", data)

### Unrelated Sentences
Should return Not Found

In [ ]:
UNRELATED = [
    "This roller coaster is very fun",
    "The hill drops into a loop",
    "The car moves along the path",
]

samples = editor.template(
    "{unrelated}.",
    unrelated = UNRELATED
)

data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Not Found"})

writeData("Knowledge", "MFT", "Insufficient definitions", data)

## Directional Tests

### Acceptable definitions followed by unacceptable definition
Acceptable -> Unacceptable

In [ ]:
samples = editor.template(
    "Potential energy is {pe_acceptable}. Potential energy is {pe_unacceptable}.",
    pe_acceptable=PE_ACCEPTABLE, pe_unacceptable=PE_UNACCEPTABLE
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_acceptable}. Kinetic energy is {ke_unacceptable}.",
    ke_acceptable=KE_ACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_acceptable}. The Law of Conservation of energy states {lce_unacceptable}.",
    lce_acceptable=LCE_ACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Knowledge", "DIR", "Correct definition followed by incorrect", data)

### Unacceptable Definition followed by acceptable definition
Unacceptable -> acceptable

In [ ]:
# code
samples = editor.template(
    "Potential energy is {pe_unacceptable}. Potential energy is {pe_acceptable}",
    pe_acceptable=PE_ACCEPTABLE, pe_unacceptable=PE_UNACCEPTABLE
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_unacceptable}. Kinetic energy is {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_unacceptable}. The Law of Conservation of energy states {lce_acceptable}",
    lce_acceptable=LCE_ACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Knowledge", "DIR", "Incorrect definition followed by correct", data)

### Insufficient definition followed by acceptable definition
Insufficient -> acceptable

In [ ]:
# code
samples = editor.template(
    "Potential energy is {pe_insufficient}. Potential energy is {pe_acceptable}",
    pe_insufficient=PE_INSUFFICIENT, pe_acceptable=PE_ACCEPTABLE
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_insufficient}. Kinetic energy is {ke_acceptable}",
    ke_insufficient=KE_INSUFFICIENT, ke_acceptable=KE_ACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_insufficient}. The Law of Conservation of energy states {lce_acceptable}",
    lce_insufficient=LCE_INSUFFICIENT, lce_acceptable=LCE_ACCEPTABLE,
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Knowledge", "DIR", "Insufficient definition followed by correct", data)

### Insufficient definition followed by unacceptable definition
Insufficient -> unacceptable

In [ ]:
samples = editor.template(
    "Potential energy is {pe_insufficient}. Potential energy is {pe_unacceptable}",
    pe_insufficient=PE_INSUFFICIENT, pe_unacceptable=PE_UNACCEPTABLE
)
data = getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_insufficient}. Kinetic energy is {ke_unacceptable}",
    ke_insufficient=KE_INSUFFICIENT, ke_unacceptable=KE_UNACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_insufficient}. The Law of Conservation of energy states {lce_unacceptable}",
    lce_insufficient=LCE_INSUFFICIENT, lce_unacceptable=LCE_UNACCEPTABLE,
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

writeData("Knowledge", "DIR", "Insufficient definition followed by incorrect", data)

# Negation

## MFTs

### Negated acceptable definition
Acceptable -> unacceptable

In [ ]:
samples = editor.template(
    "Potential energy is not {pe_acceptable}",
    pe_acceptable=PE_ACCEPTABLE
)
data = getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is not {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy does not state {lce_acceptable}.",
    lce_acceptable=LCE_ACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

writeData("Negation", "MFT", "Negation of correct definitions", data)

### Negation of unacceptable definition
unacceptable -> insufficient

In [ ]:
samples = editor.template(
    "Potential energy is not {pe_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE
)
data = getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is not {ke_unacceptable}",
    ke_unacceptable=KE_UNACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy does not state {lce_unacceptable}.",
    lce_unacceptable=LCE_UNACCEPTABLE
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})
writeData("Negation", "MFT", "Negation of incorrect definitions", data)

# Robustness

## INV

### Misspelling of words
Should return the same label

#### Acceptable definitions

In [ ]:
PE_TYPO_CORRECT = [
    "pontenshul energy that body has becuse of its posiion relative to other bdies.",
    "potental energy store in te car at the tp of the initl drp.",
]
KE_TYPO_CORRECT = [
    "Kintic enery is energy tht the car hs bcase i is movng.",
    "Kinetec enrgy is the wrk neded to acelerate the rolercoaser car from rest.",
]
LCE_TYPO_CORRECT = [
    "the lw of consevashun of energy says that energy canot be create or destroy, ony transfomed.",
    "the law of conversion of energy the totl energy of an isolated systm remain constnt.",
]

samples = editor.template(
    "{pe_typo_correct}",
    pe_typo_correct=PE_TYPO_CORRECT
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "{ke_typo_correct}",
    ke_typo_correct=KE_TYPO_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "{lce_typo_correct}.",
    lce_typo_correct=LCE_TYPO_CORRECT
)

data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Robustness", "INV", "Misspelling of correct definitions", data)

#### Unacceptable definitions

In [ ]:
PE_TYPO_INCORRECT = [
    "Potential energy i energy in moton.",
    "Potentia energy is the energy lose as th car go down the hil.",
]
KE_TYPO_INCORRECT = [
    "Kinetic enery is energy tht the car hs bcase i is movng.",
    "Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.",

]
LCE_TYPO_INCORRECT = [
    "LCE says that energy canot be create or destroy, ony transfomed.",
    "law of conserve energy state the totl energy of an isolated systm remain constnt.",
]

samples = editor.template(
    "{pe_typo_incorrect}",
    pe_typo_incorrect=PE_TYPO_INCORRECT
)
data = getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "{ke_typo_incorrect}",
    ke_typo_incorrect=KE_TYPO_INCORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "{lce_typo_incorrect}.",
    lce_typo_incorrect=LCE_TYPO_INCORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

writeData("Robustness", "INV", "Misspelling of incorrect definitions", data)

#### Insufficient definitions

In [ ]:
PE_TYPO_INSUFFICIENT = [
    "The potentia energ at the top of te rollercoaer is 4.9 juls.",
    "Potential enrgy is measur in jules.",
]
KE_TYPO_INSUFFICIENT = [
    "The kinetic energy at the botom of the hile is 4.8 joul.",
    "Kinetic nergy transfourms into heet thrugh fricton",
]
LCE_TYPO_INSUFFICIENT = [
    "The potensial energy trainsforms in to kinetic energy becuz of the law of conservashun of energy.",
    "As the car goes down the hill, some energ is lost to fricshiun as heat.",
]

samples = editor.template(
    "{pe_typo_insufficient}",
    pe_typo_insufficient=PE_TYPO_INSUFFICIENT
)
data = getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "{ke_typo_insufficient}",
    ke_typo_insufficient=KE_TYPO_INSUFFICIENT
)
data += getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "{lce_typo_insufficient}.",
    lce_typo_insufficient=LCE_TYPO_INSUFFICIENT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})

writeData("Robustness", "INV", "Misspelling of insufficient definitions", data)

### Using acronyms should not change label
Potential energy -> PE
Kinetic energy -> KE
Law of Conservation of energy -> LCE
#### Acceptable definitions
acceptable -> acceptable

In [ ]:
PE_ACRONYM_CORRECT = [
    "PE is energy at rest.",
    "PE is energy that a body has because of its position relative to other bodies.",
]
KE_ACRONYM_CORRECT = [
    "KE is energy in motion.",
    "KE is the energy that the car has because it is moving.",
]
LCE_ACRONYM_CORRECT = [
    "LCE says that energy cannot be created or destroyed, only transformed.",
    "LCE states that the total energy of an isolated system remains constant.",
]

samples = editor.template(
    "{pe_acro_correct}",
    pe_acro_correct=PE_ACRONYM_CORRECT
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "{ke_acro_correct}",
    ke_acro_correct=KE_ACRONYM_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "{lce_acro_correct}.",
    lce_acro_correct=LCE_ACRONYM_CORRECT
)

data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Robustness", "INV", "Acronyms of acceptable definitions", data)

#### Unacceptable definitions
Unacceptable -> unacceptable

In [ ]:
PE_ACRONYM_INCORRECT = [
    "PE is energy in motion.",
    "PE is the energy lost as the car goes down the hill.",
]
KE_ACRONYM_INCORRECT = [
    "Kinetic energy is energy at rest.",
    "Kinetic energy is never lost or gained as the car moves through the rollercoaster.",
]
LCE_ACRONYM_INCORRECT = [
    "The law of conservation of energy says that energy can be created and destroyed.",
    "The law of conservation of energy states that the energy of a closed system will change.",
]

samples = editor.template(
    "{pe_acro_incorrect}",
    pe_acro_incorrect=PE_ACRONYM_INCORRECT
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "{ke_acro_incorrect}",
    ke_acro_incorrect=KE_ACRONYM_INCORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "{lce_acro_incorrect}.",
    lce_acro_incorrect=LCE_ACRONYM_INCORRECT
)

data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Robustness", "INV", "Acronyms of unacceptable definitions", data)

#### Insufficient definitions
Insufficient -> Insufficient

In [ ]:
PE_ACRONYM_INSUFFICIENT = [
    "PE at the top of the rollercoaster is 4.9 joules.",
    "PE is measured in joules."
]
KE_ACRONYM_INSUFFICIENT = [
    "KE at the bottom of the hill is 4.8 joules.",
    "KE is measured in joules.",
]
LCE_ACRONYM_INSUFFICIENT = [
    "LCE states that the potential energy transforms into kinetic energy.",
]

samples = editor.template(
    "{pe_acro_insufficient}",
    pe_acro_insufficient=PE_ACRONYM_INSUFFICIENT
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "{ke_acro_insufficient}",
    ke_acro_insufficient=KE_ACRONYM_INSUFFICIENT
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "{lce_acro_insufficient}.",
    lce_acro_insufficient=LCE_ACRONYM_INSUFFICIENT
)

data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Robustness", "INV", "Acronyms of unacceptable definitions", data)

# Taxonomy

## INV
### Replacing words with synonyms
#### Acceptable definition

In [ ]:
PE_SYNONYM_CORRECT = [
    "energy that a body has because of its position relative to other bodies.",
    "energy inside the car at the top of the initial drop.",
]
KE_SYNONYM_CORRECT = [
    "energy that the vehicle has because it is moving.",
    "the work needed to change the speed the rollercoaster car from rest.",
]
LCE_SYNONYM_CORRECT = [
    "that energy can be turned from one form to another, but never created or destroyed.",
    "that if there were no rubbing, the potential energy at the top of the rollercoaster.",
    "would equal to the kinetic energy at the bottom of the drop.",
]

samples = editor.template(
    "Potential energy is {pe_synonym_correct}",
    pe_synonym_correct=PE_SYNONYM_CORRECT
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_synonym_correct}",
    ke_synonym_correct=KE_SYNONYM_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_synonym_correct}.",
    lce_synonym_correct=LCE_SYNONYM_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Taxonomy", "INV", "Synonyms within correct definitions", data)

#### Unacceptable definition

In [ ]:
PE_SYNONYM_INCORRECT = [
    "the energy lost as the car rolls down the hill.",
    "the opposite of kinetic energy.",
]
KE_SYNONYM_INCORRECT = [
    "never lost or gained as the car moves through the rollercoaster.",
    "the opposite of potential energy.",
]
LCE_SYNONYM_INCORRECT = [
    "that energy can be made and taken.",
    "that the energy of a closed system will differ.",
    "that in an open system, energy is constant",
]

samples = editor.template(
    "Potential energy is {pe_synonym_incorrect}",
    pe_synonym_incorrect=PE_SYNONYM_INCORRECT
)
data = getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_synonym_incorrect}",
    ke_synonym_incorrect=KE_SYNONYM_INCORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_synonym_incorrect}.",
    lce_synonym_incorrect=LCE_SYNONYM_INCORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

writeData("Taxonomy", "INV", "Synonyms within incorrect definitions", data)

#### Insufficient definition

In [ ]:
PE_SYNONYM_INSUFFICIENT = [
    "The potential energy at the tip of the rollercoaster is 4.9 joules.",
    "Potential energy is calculated in joules.",
]
KE_SYNONYM_INSUFFICIENT = [
    "There is higher kinetic energy at the bottom of the hill than at the top.",
    "Kinetic energy turns into heat through friction",
]
LCE_SYNONYM_INSUFFICIENT = [
    "As the car goes down the hill, some energy does away to friction as heat.",
    "If there were no friction, the energy would stay the same.",
]

samples = editor.template(
    "{pe_synonym_insufficient}",
    pe_synonym_insufficient=PE_SYNONYM_INSUFFICIENT
)
data = getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "{ke_synonym_insufficient}",
    ke_synonym_insufficient=KE_SYNONYM_INSUFFICIENT
)
data += getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "{lce_synonym_insufficient}.",
    lce_synonym_insufficient=LCE_SYNONYM_INSUFFICIENT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})

writeData("Taxonomy", "INV", "Synonyms within insufficient definitions", data)

### Replacing words with negated antonym
Label should remain the same
#### Acceptable definitions
Acceptable -> Acceptable

In [ ]:
PE_NEGANT_CORRECT = [
    "energy that a body does not lack because of its position relative to other bodies.",
    "energy not outside the car at the top of the initial drop.",
]
KE_NEGANT_CORRECT = [
    "energy that the car has because it is not still.",
    "the work needed to change the speed the rollercoaster car from not moving.",
]
LCE_NEGANT_CORRECT = [
    "that energy can be transformed from one form to another, but not ever created or destroyed.",
    "that the total energy of an isolated system remains not variable.",
]

samples = editor.template(
    "Potential energy is {pe_negant_correct}",
    pe_negant_correct=PE_NEGANT_CORRECT
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_negant_correct}",
    ke_negant_correct=KE_NEGANT_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_negant_correct}.",
    lce_negant_correct=LCE_NEGANT_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

writeData("Taxonomy", "INV", "Negated antonyms within correct definitions", data)

#### Unacceptable definitions
Unacceptable -> unacceptable

In [ ]:
PE_NEGANT_INCORRECT = [
    "energy not stationary.",
    "not the same as kinetic energy.",
]
KE_NEGANT_INCORRECT = [
    "energy not in motion.",
    "not the same as potential energy.",
]
LCE_NEGANT_INCORRECT = [
    "that the energy of a not open system will change.",
    "that in a not closed system, energy is conserved",
]

samples = editor.template(
    "Potential energy is {pe_negant_incorrect}",
    pe_negant_incorrect=PE_NEGANT_INCORRECT
)
data = getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_negant_incorrect}",
    ke_negant_incorrect=KE_NEGANT_INCORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_negant_incorrect}.",
    lce_negant_incorrect=LCE_NEGANT_INCORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

writeData("Taxonomy", "INV", "Negated antonyms within incorrect definitions", data)

# Fairness

## INV
## Test for words being replaced in Spanish
#### Acceptable definitions

In [ ]:
PE_SPANISH_CORRECT = [
    "Potential energy is energy that a body has because of its position relativa a other bodies.",
    "Potential energy is the stored energy of the rollercoaster car when no se esta moviendo.",
    "Potential energy is energy that has the potential to convertirse en otro form of energy.",
]
KE_SPANISH_CORRECT = [
    "Kinetic energy is the work necesario para accelerate the rollercoaster car from rest.",
    "Kinetic energy is determined by the mass del carro and the velocity with which it is moving.",
    "Kinetic energy quantifies the work an object performs debido a su movimiento",
]
LCE_SPANISH_CORRECT = [
    "LCE states that energy can be convertida from one form to another, but never created or destroyed.",
    "LCE says that if there were no friction, the potential energy en la cima del rollercoaster would be the same as the kinetic energy at the bottom of the drop.",
    "LCE is a physical law that states that energy no se puede crear or destroyed but only transformed",
]

samples = editor.template(
    "Potential energy is {pe_spanish_correct}",
    pe_spanish_correct=PE_SPANISH_CORRECT,
)
data = getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_spanish_correct}",
    ke_spanish_correct=KE_SPANISH_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_spanish_correct}.",
    lce_spanish_correct=LCE_SPANISH_CORRECT
)
data += getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})
writeData("Fairness", "INV", "Spanish translation within correct definitions", data)